# Set up example

In [2]:
import pandas as pd
import numpy as np
#Input Variables for trainalgorithm
address = '516 W 8th Street Austin TX'
categories = ['4bf58dd8d48988d1e2931735',
        '4deefb944765f83613cdba6e',
        '4bf58dd8d48988d142941735',
        '4bf58dd8d48988d1d0941735',
        '4bf58dd8d48988d159941735']
D = 300
N = 4
#Additional Input Variables for tunealgorithm and testalgorithm
start = [30.2713320408163, -97.7469014285714]
venues = pd.read_csv('venuesE1.csv', index_col=0)
#Convert location column in venues dataframe from string
i = 0
for v in venues['location']:
    venues['location'][i] = eval(v)
    i += 1


/home/jupyterlab/conda/lib/python3.6/site-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


## Direct Solution

In [3]:
OLoop, BL = optimalloop(start, venues, categories, D, N)
OLoop.to_csv("OLoopE1.csv")
%store BL

NameError: name 'optimalloop' is not defined

## Analysis

In [46]:
epsilon_ = 0.1
gamma_ = .9
its = 5
N_steps = 1500
 
var = pd.DataFrame(columns=['L', 'BL'])

filepre = 'E1Loop' + str(N_steps)+ 'e' + str(int(epsilon_*100)) + 'g' + str(int(gamma_*100))
detfilepre = 'E1Loop_det' + str(N_steps)+ 'e' + str(int(epsilon_*100)) + 'g' + str(int(gamma_*100))
vfile = 'E1vars' + str(N_steps)+ 'e' + str(int(epsilon_*100)) + 'g' + str(int(gamma_*100)) + '.csv'
for i in range(1, its+1):
    filesuff = '_' + str(i) + '.csv'
    file = filepre + filesuff
    detfile = detfilepre + filesuff
    Loop_det, Loop, L, BL= tunealgorithm(start, venues, categories, D, N, epsilon_, gamma_, N_steps)
    row = 'i=' + str(i)
    var.loc[row] = [L, BL]
    Loop.to_csv(file)
    Loop_det.to_csv(detfile)
var.to_csv(vfile)

In [ ]:
epsilon_ = 0.1
gamma_ = 1
its = 30
MAX = np.percentile(OLoop['Value'], 99)
 
var = pd.DataFrame(columns=['L', 'BL'])

filepre = 'E1LoopMAXe' + str(int(epsilon_*100)) + 'g' + str(int(gamma_*100))
detfilepre = 'E1Loop_detMAXe' + str(int(epsilon_*100)) + 'g' + str(int(gamma_*100))
vfile = 'E1varsMAXe' + str(int(epsilon_*100)) + 'g' + str(int(gamma_*100)) + '_4.csv'
for i in range(31, its+31):
    filesuff = '_' + str(i) + '.csv'
    file = filepre + filesuff
    detfile = detfilepre + filesuff
    Loop_det, Loop, L, BL= testalgorithm(start, venues, categories, D, N, MAX, epsilon_, gamma_)
    row = 'i=' + str(i)
    var.loc[row] = [L, BL]
    Loop.to_csv(file)
    Loop_det.to_csv(detfile)
var.to_csv(vfile)

In [19]:
epsilon_ = 0.1
gamma_ = 1
its = 30
MAX = np.percentile(OLoop['Value'], 95)
 
var = pd.DataFrame(columns=['L', 'BL'])

filepre = 'E1LoopMAX95e' + str(int(epsilon_*100)) + 'g' + str(int(gamma_*100))
detfilepre = 'E1Loop_detMAX95e' + str(int(epsilon_*100)) + 'g' + str(int(gamma_*100))
vfile = 'E1varsMAX95e' + str(int(epsilon_*100)) + 'g' + str(int(gamma_*100)) + '.csv'
for i in range(1, its+1):
    filesuff = '_' + str(i) + '.csv'
    file = filepre + filesuff
    detfile = detfilepre + filesuff
    Loop_det, Loop, L, BL= testalgorithm(start, venues, categories, D, N, MAX, epsilon_, gamma_)
    row = 'i=' + str(i)
    var.loc[row] = [L, BL]
    Loop.to_csv(file)
    Loop_det.to_csv(detfile)
var.to_csv(vfile)

# Functions

OLoop, BL = optimalloop(start, venues, categories, D, N)

In [2]:
def optimalloop(start, venues, categories, D, N):
    import pandas as pd  
    distance = getdistances(venues)
    state0 = {'number': 0, 'location': start, 'venues': ['']*N, 'categories':['']*N}#Initial state
    BL = 0
    #All possible first states
    available1 = getactionchoices(state0, N, D, venues, distance)
    state1 = pd.DataFrame(columns=['state', 'reward'])
    for i in range(len(available1)):
        state1.loc[i] = ''
        a = available1.loc[i]
        dist, dr, cr, r = getreward(state0, a, D, N, distance)
        state1.iloc[i, 0] = [changestate(state0, a)]
        state1.iloc[i, 1] = r
    print(state1.shape)
    #All possible second states
    state2 = pd.DataFrame(columns=['state', 'reward'])
    i2 = 0
    for s1 in range(len(state1)):
        s = state1.loc[s1]
        available2 = getactionchoices(s['state'][0], N, D, venues, distance)
        if available2 is not None:
            for i in range(len(available2)): 
                state2.loc[i2] = ''
                a = available2.loc[i]
                dist, dr, cr, r = getreward(s['state'][0], a, D, N, distance)
                state2.iloc[i2, 0] = [changestate(s['state'][0], a)]
                state2.iloc[i2, 1] = s['reward'] + r
                i2 += 1
        else: BL += 1
    print(state2.shape)
    #All possible third states
    state3 = pd.DataFrame(columns=['state', 'reward'])
    i3 = 0
    for s2 in range(len(state2)):
        s = state2.loc[s2]
        available3 = getactionchoices(s['state'][0], N, D, venues, distance)
        if available3 is not None:
            for i in range(len(available3)): 
                state3.loc[i3] = ''
                a = available3.loc[i]
                dist, dr, cr, r = getreward(s['state'][0], a, D, N, distance)
                state3.iloc[i3, 0] = [changestate(s['state'][0], a)]
                state3.iloc[i3, 1] = s['reward'] + r
                i3 += 1
        else: BL += 1
    print(state3.shape)
    #All possible fourth states
    state4 = pd.DataFrame(columns=['state', 'reward'])
    i4 = 0
    for s3 in range(len(state3)):
        s = state3.loc[s3]
        available4 = getactionchoices(s['state'][0], N, D, venues, distance)
        if available4 is not None:
            for i in range(len(available4)): 
                state4.loc[i4] = ''
                a = available4.loc[i]
                dist, dr, cr, r = getreward(s['state'][0], a, D, N, distance)
                state4.iloc[i4, 0] = [changestate(s['state'][0], a)]
                state4.iloc[i4, 1] = s['reward'] + r
                i4 += 1
        else: BL += 1
    print(state4.shape)
    #Copy, paste, and change for loop if N > 4, comment lines if N < 4
    
    stops = ['']*N
    for stp in range(1, N+1):
        stops[stp-1] = 'Stop ' + str(stp)
    col = ['Value'] + stops
    OLoop = pd.DataFrame(columns=col)
    
    #Change so stateL = last state
    stateL = state4
    for i in range(len(stateL)):
        s = stateL.loc[i]
        OLoop.loc[i] = ''
        OLoop.iloc[i, 0] = s['reward']
        for n in range(1, N+1):
            OLoop.iloc[i, n] = s['state'][0]['venues'][n-1]
        
    OLoop.sort_values(['Value'], ascending=False, inplace=True)
    return OLoop, BL

Loop_det, Loop, L, BL = trainalgorithm(address, max_dist, num_stops, categories)

In [3]:
def trainalgorithm(address, D, N, categories):
    #start: address of starting location
    #D: max distance to travel between stops
    #N: number of stops to make
    #categories: category ID for the categories the user is interested in
    
    import pandas as pd
    import numpy as np
    from geopy.geocoders import Nominatim 
    import geopy.distance
    epsilon = 0.3
    gamma = 0.9
    N_steps = 500
    geopy.geocoders.options.default_user_agent = "my-application"
    location = Nominatim().geocode(address)
    lat = location.latitude
    lng = location.longitude
    start = [lat, lng] 
    venues = get_all_venues(D, N, categories, start)
    distance = getdistances(venues)
    
    q = {}
    state = {'number': 0, 'location': start, 'venues': ['']*N, 'categories':['']*N}
    action = None
    
    Loop_det = pd.DataFrame(columns=(['Loop', 'Loop Value', 'Name', 'Location', 'Category', 'Category ID', 'Distance from Last',
                                 'c_reward', 'd_reward', 'total_reward']))
    loop = pd.DataFrame(columns=(['Loop', 'Loop Value', 'Name', 'Location', 'Category', 'Category ID', 'Distance from Last',
                                 'c_reward', 'd_reward', 'total_reward']))
    
    stops = ['']*N
    for stp in range(1, N+1):
        stops[stp-1] = 'Stop ' + str(stp)
    col = ['Loop', 'Value'] + stops
    Loop = pd.DataFrame(columns=col)
    L = 1
    l = 0
    step = 0
    BL = 0
    while step < N_steps:
        Loop.loc[L-1] = ''
        if state['number'] == 0:
            action = chooseaction(state, N, D, venues, epsilon, q, distance)
            reward = assigndr(action[4], D)
            loop.loc[0] = [L, '', action[0], action[1], action[2], action[3], action[4], 0, reward, reward]
            learnq(state, action, N, D, venues, q, distance, gamma)
            state = changestate(state, action)
        elif state['number'] == N:
            valuef = sum(loop['total_reward'])
            loop.iloc[:, 1] = valuef            
            Loop_det = Loop_det.append(loop)
            Loop.iloc[L-1, 0:2] = L, valuef
            for v in range(N):
                Loop.iloc[L-1, v+2] = loop.iloc[v,2]
            L += 1
            state = {'number': 0, 'location': start, 'venues': ['']*N, 'categories':['']*N}
            loop = loop.iloc[0:0]
        else:
            action = chooseaction(state, N, D, venues, epsilon, q, distance)
            if action is None:
                state = {'number': 0, 'location': start, 'venues': ['']*N, 'categories':['']*N}
                loop = loop.iloc[0:0]
                BL += 1
            else:
                dist, d_reward, c_reward, total_reward = getreward(state, action, D, N, distance)
                loop.loc[state['number']] = [L, '', action[0], action[1], action[2], action[3], dist, c_reward, d_reward, total_reward]
                learnq(state, action, N, D, venues, q, distance, gamma)
                state = changestate(state, action)
        step += 1
    Loop_det.reset_index(inplace=True, drop=True)
    L = L-1
    return Loop_det, Loop, L, BL

Loop_det, Loop, L, BL = tunealgorithm(start, venues, categories, D, N, epsilon_=0.1, gamma_=0.9, N_steps=50)

In [4]:
def tunealgorithm(start, venues, categories, D, N, epsilon_=0.1, gamma_=0.9, N_steps=500):
    #start: address of starting location
    #D: max distance to travel between stops
    #N: number of stops to make
    #categories: category ID for the categories the user is interested in
    
    import pandas as pd
    import numpy as np
    import geopy.distance
    epsilon = epsilon_
    gamma = gamma_

    distance = getdistances(venues)
    
    q = {}
    state = {'number': 0, 'location': start, 'venues': ['']*N, 'categories':['']*N}
    action = None
    
    Loop_det = pd.DataFrame(columns=(['Loop', 'Loop Value', 'Name', 'Location', 'Category', 'Category ID', 'Distance from Last',
                                 'c_reward', 'd_reward', 'total_reward']))
    loop = pd.DataFrame(columns=(['Loop', 'Loop Value', 'Name', 'Location', 'Category', 'Category ID', 'Distance from Last',
                                 'c_reward', 'd_reward', 'total_reward']))
    
    stops = ['']*N
    for stp in range(1, N+1):
        stops[stp-1] = 'Stop ' + str(stp)
    col = ['Loop', 'Value'] + stops
    Loop = pd.DataFrame(columns=col)
    L = 1
    l = 0
    step = 0
    BL = 1
    while step < N_steps:
        if state['number'] == 0:
            action = chooseaction(state, N, D, venues, epsilon, q, distance)
            reward = assigndr(action[4], D)
            loop.loc[0] = [L, '', action[0], action[1], action[2], action[3], action[4], 0, reward, reward]
            learnq(state, action, N, D, venues, q, distance, gamma)
            state = changestate(state, action)
        elif state['number'] == N:
            valuef = sum(loop['total_reward'])
            loop.iloc[:, 1] = valuef            
            Loop_det = Loop_det.append(loop)
            Loop.loc[l] = ''
            Loop.iloc[l, 0:2] = L, valuef
            for v in range(N):
                Loop.iloc[l, v+2] = loop.iloc[v,2]
            L += 1
            l += 1
            state = {'number': 0, 'location': start, 'venues': ['']*N, 'categories':['']*N}
            loop = loop.iloc[0:0]
        else:
            action = chooseaction(state, N, D, venues, epsilon, q, distance)
            if action is None:
                Loop_det = Loop_det.append(loop)
                Loop.loc[l] = ''
                Loop.iloc[l, 0:2] = BL, 0
                for v in range(len(loop)):
                    Loop.iloc[l, v+2] = loop.iloc[v,2]
                state = {'number': 0, 'location': start, 'venues': ['']*N, 'categories':['']*N}
                loop = loop.iloc[0:0]
                BL += 1
                l += 1
            else:
                dist, d_reward, c_reward, total_reward = getreward(state, action, D, N, distance)
                loop.loc[state['number']] = [L, '', action[0], action[1], action[2], action[3], dist, c_reward, d_reward, total_reward]
                learnq(state, action, N, D, venues, q, distance, gamma)
                state = changestate(state, action)
        step += 1
    Loop_det.reset_index(inplace=True, drop=True)
    L = L - 1 # So that L represents the number of loops
    BL = BL - 1
    return Loop_det, Loop, L, BL

Loop_det, Loop, L, BL = testalgorithm(start, venues, categories, D, N, MAX, epsilon_=0.1, gamma_=0.9)

In [5]:
def testalgorithm(start, venues, categories, D, N, MAX, epsilon_=0.1, gamma_=0.9):
    #start: address of starting location
    #D: max distance to travel between stops
    #N: number of stops to make
    #categories: category ID for the categories the user is interested in
    
    import pandas as pd
    import numpy as np
    import geopy.distance
    epsilon = epsilon_
    gamma = gamma_

    distance = getdistances(venues)
    
    q = {}
    state = {'number': 0, 'location': start, 'venues': ['']*N, 'categories':['']*N}
    action = None
    
    Loop_det = pd.DataFrame(columns=(['Loop', 'Loop Value', 'Name', 'Location', 'Category', 'Category ID', 'Distance from Last',
                                 'c_reward', 'd_reward', 'total_reward']))
    loop = pd.DataFrame(columns=(['Loop', 'Loop Value', 'Name', 'Location', 'Category', 'Category ID', 'Distance from Last',
                                 'c_reward', 'd_reward', 'total_reward']))
    
    stops = ['']*N
    for stp in range(1, N+1):
        stops[stp-1] = 'Stop ' + str(stp)
    col = ['Loop', 'Value'] + stops
    Loop = pd.DataFrame(columns=col)
    L = 1
    l = 0
    step = 0
    BL = 1
    Loop.loc[l] = [0, 0, '', '', '', '']
    while Loop['Value'].max() < MAX:
        if state['number'] == 0:
            action = chooseaction(state, N, D, venues, epsilon, q, distance)
            reward = assigndr(action[4], D)
            loop.loc[0] = [L, '', action[0], action[1], action[2], action[3], action[4], 0, reward, reward]
            learnq(state, action, N, D, venues, q, distance, gamma)
            state = changestate(state, action)
        elif state['number'] == N:
            valuef = sum(loop['total_reward'])
            loop.iloc[:, 1] = valuef            
            Loop_det = Loop_det.append(loop)
            Loop.loc[l] = ''
            Loop.iloc[l, 0:2] = L, valuef
            for v in range(N):
                Loop.iloc[l, v+2] = loop.iloc[v,2]
            L += 1
            l += 1
            state = {'number': 0, 'location': start, 'venues': ['']*N, 'categories':['']*N}
            loop = loop.iloc[0:0]
        else:
            action = chooseaction(state, N, D, venues, epsilon, q, distance)
            if action is None:
                Loop_det = Loop_det.append(loop)
                Loop.loc[l] = ''
                Loop.iloc[l, 0:2] = BL, 0
                for v in range(len(loop)):
                    Loop.iloc[l, v+2] = loop.iloc[v,2]
                state = {'number': 0, 'location': start, 'venues': ['']*N, 'categories':['']*N}
                loop = loop.iloc[0:0]
                BL += 1
                l += 1
            else:
                dist, d_reward, c_reward, total_reward = getreward(state, action, D, N, distance)
                loop.loc[state['number']] = [L, '', action[0], action[1], action[2], action[3], dist, c_reward, d_reward, total_reward]
                learnq(state, action, N, D, venues, q, distance, gamma)
                state = changestate(state, action)
        step += 1
    Loop_det.reset_index(inplace=True, drop=True)
    L = L - 1 # So that L represents the number of loops
    BL = BL - 1
    return Loop_det, Loop, L, BL

venues = get_all_venues(D, N, categories, start)

In [6]:
#Get all venues that may possibly be selected
def get_all_venues(D, N, categories, start):
    import pandas as pd
    radius = D*(N/2) #search radius assuming one can not go furhter than this and still loop back to the hotel
    venues = pd.DataFrame(columns=['name', 'location', 'category', 'category ID', 'distance from start'])
    for i in range(len(categories)):
        venues = venues.append(get_venues_in_category(start, categories[i], radius))
    venues.reset_index(inplace=True, drop=True)
    return venues

search_df = get_venues_in_category(location, category, radius, limit=50)

In [7]:
def get_venues_in_category(location, category, radius, limit=50):
    #Define Foursquare credintials
    CLIENT_ID = '4SZ3RB4ZDJIJJ1PC5QZHP5HT1AKMTPLWCJIDH53UZUMRKWJJ' # your Foursquare ID
    CLIENT_SECRET = 'HKRGO3VZQTW0E332ZTVVAPYO2XGT3PR2VLAY2DWTZIEQDPDE' # your Foursquare Secret
    VERSION = '20180605' # Foursquare API version

    # create the API request URL
    import pandas as pd
    import requests
    lat = location[0]
    lng = location[1]
    url = 'https://api.foursquare.com/v2/venues/search?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}&categoryId={}&intent=browse'.format(
       CLIENT_ID, 
       CLIENT_SECRET, 
       VERSION, 
       lat, 
       lng, 
       radius, 
       limit,
       category)
    results = requests.get(url).json()['response']['venues']
    search_df = pd.DataFrame(columns=['name', 'location', 'category', 'category ID', 'distance from start'])
    for i in range(len(results)):
        search_df.loc[i] = ''
        search_df.iloc[i,0] = results[i]['name']
        search_df.iloc[i,1] = [results[i]['location']['lat'], results[i]['location']['lng']]
        search_df.iloc[i,2] = results[i]['categories'][0]['name']
        search_df.iloc[i,3] = category
        search_df.iloc[i,4] = results[i]['location']['distance']
    return search_df

distance = getdistances(venues)

In [8]:
def getdistances(venues):
    import pandas as pd
    import geopy.distance
    distance = pd.DataFrame(columns=venues['name'], index=venues['name'])
    for r in range(len(venues)):
        loc1 = venues['location'][r]
        for c in range(len(venues)):
            loc2 = venues['location'][c] 
            d = geopy.distance.distance(loc1, loc2).m
            distance.iloc[r, c] = d
    return distance

dr = assigndr(d, D)

In [9]:
def assigndr(d, D):
    if d > D:
        dr = -100
    elif d == 0:
        dr = -100
    else: 
        dr = 5*((D - d)/D) + 5
    return dr

a2 = chooseaction(s, N, D, venues, epsilon, q, distance)

In [10]:
def chooseaction(s, N, D, venues, epsilon, q, distance):
    import random
    available = getactionchoices(s, N, D, venues, distance)
    if available is None:
        a2 = None
    else:
        if random.random() < epsilon:
            i = random.choice(available.index)
            a2 = available.loc[i]
        else:
            qs = [''] * len(available)
            ind = 0
            for a in available['name']:
                qs[ind] = q.get((tuple(s['venues']), a), 0.0)
                ind += 1
            maxQ = max(qs)
            count = qs.count(maxQ)
            if count > 1:
                best = [i for i in range(len(available)) if qs[i] == maxQ]
                i = random.choice(best)
            else:
                i = qs.index(maxQ)

            a2 = available.loc[i]

    return a2   

available = getactionchoices(s, N D, venues, distance)

In [11]:
def getactionchoices(s, N, D, venues, distance):
    #Get all venues in the radius of the current venue 
    #that we havent visited and 
    #where the start can still be reached 
    import pandas as pd
    pd.options.mode.chained_assignment = None
    d_remain = ((N+1) - s['number']) * D
    
    if s['number'] == 0:
        #If at the start, available is all venues within the radius of start
        available = venues[venues.iloc[:,4] <= D]
        available.reset_index(inplace=True, drop=True)

    elif s['number'] == N-1:
        #Venues close enough to start to be able to loop back
        close_start = venues[venues.iloc[:,4] <= D]
        #Venues within radius of current location
        cl = s['venues'][s['number']-1]
        close_cl = distance[distance.loc[cl] <= D][cl]
        cind = close_cl[close_cl == 0].index
        cind = cind[0]
        close_cl.drop(index=cind, inplace=True)

        #Venues that meet both proximity requirements        
        close = close_start.iloc[0:0]
        for i in close_start['name']:
            if i in close_cl.index:
                ind = close_start[close_start['name'] == i].index
                ind = ind[0]
                close.loc[ind] = close_start.loc[ind]
        #Venues not already on the loop
        for i in s['venues']:
            for c in close['name']:
                if i == c:
                    ind = close[close['name'] == i].index
                    ind = ind[0]
                    close.drop(index=ind, inplace=True)
        available = close.reset_index(drop=True)
    else:
        #Venues within radius of current location
        cl = s['venues'][s['number']-1]
        close_cl = distance[distance.loc[cl] <= D][cl]
        cind = close_cl[close_cl == 0].index
        cind = cind[0]
        close_cl.drop(index=cind, inplace=True)
        #Venues close enough to start to be able to loop back
        close_start = venues[venues.iloc[:,4] <= d_remain]
        #Venues that meet both proximity requirements
        close = close_start.iloc[0:0]        
        for i in close_start['name']:
            if i in close_cl.index:
                ind = close_start[close_start['name'] == i].index
                ind = ind[0]
                close.loc[ind] = close_start.loc[ind]
        #Venues not already on the loop
        for i in s['venues']:
            for c in close['name']:
                if i == c:
                    ind = close[close['name'] == i].index
                    ind = ind[0]
                    close.drop(index=ind, inplace=True)
        available = close.reset_index(drop=True)
    if len(available) == 0:
        available = None
    return available

s2 = changestate(s, a)

In [12]:
def changestate(s=None,a=None):
    import copy
    a_loc = a['location']
    s2 = copy.deepcopy(s)

    s2['location'] = a_loc
    s2['venues'][s2['number']] = a['name']
    s2['categories'][s2['number']] = a['category ID']
    s2['number'] += 1

    return s2

learnq(s, a, N, D, venues, q, distance, gamma)

In [13]:
def learnq(s, a, N, D, venues, q, distance, gamma):
    oldv = q.get((tuple(s['venues']), a['name']), None)
    s2 = changestate(s, a)
    dist, d_reward, c_reward, total_reward = getreward(s, a, D, N, distance)
    if oldv is None:
        q[(tuple(s['venues']), a['name'])] = total_reward
    else:
        #If there are future actions to take from the next state
        if s['number'] < N-1:
            available2 = getactionchoices(s2, N, D, venues, distance)
            if available2 is None:
                q[(tuple(s['venues']), a['name'])] = -100
            else:
                maxqnew = max([q.get((tuple(s2['venues']), a2), 0.0) for a2 in available2['name']])
                q[(tuple(s['venues']), a['name'])] = total_reward + gamma*maxqnew
        else:
            #If there are no next actions to take
            q[(tuple(s['venues']), a['name'])] = total_reward



dist, dr, cr, reward = getreward(s, a, D, N, distance)

In [14]:
def getreward(s, a, D, N, distance): #s contains current location, a is new location
    #Get the current location from s
    dfs = a['distance from start']   
    if s['number'] != 0:
        cl = s['venues'][s['number']-1] #curent location
        dist = distance[cl][a['name']]
        #distance reward
        dr = assigndr(dist, D)

        #category reward
        dnc = N - len(categories) #difference between number of stops and number of categories
        a_cat = a['category ID'] #action category
        cc = s['categories'][s['number']-1] #current category
        cat_rep = [''] * len(s['categories'])
        for i in range(len(s['categories'])):
            cat_rep[i] = s['categories'][i] == a_cat
        cr = (1 - sum(cat_rep)/N)*10
        if a_cat == cc:
            cr = cr - 5
           
        #finishing reward
        if s['number'] == N-1:
            fr = 2*assigndr(dfs, D)
        else:
            fr = 0
    else:
        dist = dfs
        dr = assigndr(dfs, D)
        cr = 0
        fr = 0
    reward = dr + cr + fr
    
    return dist, dr, cr, reward